## Intro to Path Integral

#### Muhammad Gaffar

-------------------------------------

Often, it is impossible to compute the convolution of density matrix analytically. Also, Computer is hardly can compute the matrix squaring with reasonably discretized $x$. The idea of quantum monte carlo becomes relevant here, it led to feynman path integral.

Instead, evaluation the convolution operation one of after another, we can express the density matrix in the chunk of integral of higher temperature density of matrix

$$
\rho(x_0,x_N,\beta) = \int \dots \int dx_1 \dots dx_{N-1} \;\; \underbrace{\rho\left(x_0,x_1,\frac{\beta}{N}\right) \dots \rho\left(x_{N-1},x_{N},\frac{\beta}{N}\right)}_{\text{sequence of weight for each}\; x_i}
$$

for partition function, there are more integration over the variable $x_0$

$$
Z = \int dx_0 \; \rho(x_0,x_0,\beta) = \int \dots \int dx_0 \dots dx_{N-1} \;\; \rho\left(x_0,x_1,\frac{\beta}{N}\right) \dots \rho\left(x_{N-1},x_{0},\frac{\beta}{N}\right)
$$

the sequence of $\lbrace x_0,...,x_N \rbrace$ is called path. Density matrices and partition function are thus represented in path integral.

### Path Sampling

#### A. Naive Sampling

Let us sample of path contributing to partition function, using harmonic oscillator potential

$$
V(x) = \frac{1}{2}m\omega^2 x^2 \equiv \frac{1}{2}x^2
$$

using Trotter formula, the weight of each path of $x_k$ is given by

$$
\rho_0(x_{k-1},x_k,\Delta_\tau)e^{-\frac{1}{2}\Delta_\tau V(x_k)} e^{-\frac{1}{2}\Delta_\tau V(x_k)}\rho_0(x_{k},x_{k+1},\Delta_\tau) = \rho_0(x_{k-1},x_k,\Delta_\tau)\rho_0(x_{k},x_{k+1},\Delta_\tau)e^{-\frac{1}{2}\Delta_\tau x_k^2}
$$

Using markov chain monte carlo, we see that from above, the move probability of $x_k$ is determined by $x_{k-1}$ and $x_{k+1}$

Now, we gonna use the path integral to compute the $\pi(x) = \rho(x,x,\beta)$ and compare it to analytically exact harmonic oscillator potential, where it can be derived to get

$$
\rho(x,x,\beta) = \frac{1}{\sqrt{2\pi\sinh(\beta)}}\exp\left[-x^2\tanh\left(\frac{\beta}{2}\right)\right]
$$

In [ ]:
"""
Free density matrix
"""
function ρ0(x1,x2,β)
    A = 1 / sqrt(2π * β)
    B = exp(- (x1-x2)^2 / 2β)
    return A * B
end

"""
ρ(x,x,β) of harmonic oscillator potential, where V(x) = 1/2 * x^2
"""
function ρ_diag_harmonic(x,β)
    A = 1/(2π*sinh(β))
    B = exp(-x^2 * tanh(β/2))
    return sqrt(A) * B
end

"""
Markov chain monte carlo procedure

1. Generate initial path, x = {x1,x2,...,xN}
2. Imaginary time increment, Δτ = β / N
3. choice x index and the index of its neighbors, 
        k  = rand(1,N)
        k+ = k + 1
        k- = k - 1
        if k- = 0 then k- => N + 1 # make sure the periodicity of path
4. move x[k]
        x_old = x[k]
        x_new = x[k] + rand(-δ,δ)
5. accept or reject the x_new ?
        π(x_old) = ρ(x-,x_old,Δτ) * ρ(x_old,x+,Δτ) * exp(-x_old^2 / 2)
        π(x_new) = ρ(x-,x_new,Δτ) * ρ(x_new,x+,Δτ) * exp(-x_new^2 / 2)
   accept if rand(0,1) < π(x_new) / π(x_old)
        x[k] = x_new

6. save the acceptance of path position
"""
function naive_harmonic(β,N,niter;x_in=nothing)
    #initialization
    Δτ = β/N
    if x_in == nothing
        x = zeros(Float64,N+1)
    else
        x = x_in
    end
    
    hist = []
    
    #start markov chain monte carlo
    for i in 1:niter
        k = rand(1:N)
        kp = k + 1
        km = k - 1
        if km == 0
            km = N + 1
        end

        xk_move = x[k] + (rand()-0.5)
        
        πa = ρ0(x[km],x[k],Δτ) * ρ0(x[k],x[kp],Δτ) * exp(-0.5*Δτ*x[k]^2)
        πb = ρ0(x[km],xk_move,Δτ) * ρ0(xk_move,x[kp],Δτ) * exp(-0.5*Δτ*xk_move^2)
        γ  = πb / πa
        if rand() < γ
            x[k] = xk_move
            push!(hist,x[k])
        end
        x[N] = x[1]
    end
    
    return hist,x
end;

In [ ]:
using Plots

p = plot(framestyle=:box,size=(400,500))
xlims!(p,-2,2)

x = zeros(9)
plot!(p,x,0:length(x)-1,marker=:circle,label = "iter = 0")

β = 4
N = 8
for i in 1:10
    global x
    hist,x = naive_harmonic(β,N,1,x_in=x)
    plot!(p,x,0:length(x)-1,marker=:circle,label = "iter = $i")
    xlims!(p,(-2,2))
    ylims!(p,(0,length(x)-1))
end
p

We compare the monte carlo result with exact result

In [ ]:
using NumericalIntegration

#exact diagonal density matrix harmonic
β = 4
x_path = LinRange(-3,3,100)
y = ρ_diag_harmonic.(x_path,β)
norm = integrate(x_path,y)
y ./= norm

#using monte carlo
N = 10
hst,x = naive_harmonic(β,N,1e6);

#plot
p = plot(size=(500,400))
plot!(p,x,hst,seriestype=:stephist,normalize=true,label="montecarlo")
plot!(p,x_path,y,label="exact")
ylims!(p,0,0.7)
ylabel!(p,"\\pi (x)")
xlabel!(p,"x")

Path sampling above is local sampling, and it is slow (that's why called naive). A position $x_k$ cannot get far away from $x_{k-1}$ and $x_{k+1}$, thus the computed free density matrix is becomes very small. Local path sampling is unfit for complicated problems.

#### B. Direct Sampling (Levy Sampling)

To tackle the high rejection rate sampling in local sampling, we need to construct a free rejection sampling, one of the method is levy construction.

First, to know the idea behind levy construction, let us sample the free density matrix

$$
\rho_0 (x_0,x_N,\beta) = \int \dots \int dx_1 \dots dx_{N-1} \; \rho_0(x_0,x_1,\Delta\tau)\rho_0(x_1,x_2,\Delta\tau)\dots \rho_0(x_{N-1},x_{N},\Delta\tau)
$$

we focus on $x_k$ that sandwhiched between $x'$ and $x''$ with two intervals $\Delta\tau'$ and $\Delta\tau''$, The distrituion of accepted moves is given by

$$
\pi_0 (x_k | x',x'') \propto \rho_0(x',x_k,\Delta\tau')\rho_0(x_k,x'',\Delta\tau'') 
 \propto \exp\left(-\frac{x'^2-2x'x_k+x_k^2}{2\Delta\tau'} - \frac{x_k^2-2x_kx''+x''^2}{2\Delta\tau''}\right)
$$

dropping the terms that independent to $x_k$ (we just need to know the relation to $x_k$, we can do proportional)

$$
\pi_0 (x_k | x',x'') \propto \exp\left[-\left(\frac{-x'x_k+\frac{1}{2}x_k^2}{\Delta\tau'} + \frac{-x_kx''+\frac{1}{2}x_k^2}{\Delta\tau''}\right) \right] \propto \exp\left[-\left(\frac{x_k^2 (\Delta\tau' + \Delta\tau'')}{2\Delta\tau'\Delta\tau} - \frac{x_k(x'\Delta\tau''+x''\Delta\tau')}{\Delta\tau'\Delta\tau''}\right) \right]
$$


this is again simply gaussian if we look at this way

$$
\pi_0 (x_k | x',x'') \propto \exp\left[-\left(\frac{x_k^2 (\Delta\tau' + \Delta\tau'')}{2\Delta\tau'\Delta\tau} - \frac{x_k(x'\Delta\tau''+x''\Delta\tau')}{\Delta\tau'\Delta\tau''}\frac{\Delta\tau' + \Delta\tau''}{\Delta\tau' + \Delta\tau''}\right) \right]\\ 
\propto \exp\left[-\left( x_k^2 \frac{1}{2}\left(\frac{1}{\Delta\tau'} + \frac{1}{\Delta\tau''}\right) - x_k\frac{(x'\Delta\tau''+x''\Delta\tau')}{\Delta\tau' + \Delta\tau''}\left(\frac{1}{\Delta\tau'} + \frac{1}{\Delta\tau''}\right) + C(x,x'') \right) \right]
$$

in simpler notation

$$
\pi_0 (x_k | x',x'') \propto \exp\left[ - \frac{(x_k - \langle x_k \rangle)^2}{2\sigma^2} \right]
$$

where 
$$
\langle x_k \rangle = \frac{\Delta\tau''x' + \Delta\tau'x''}{\Delta\tau' + \Delta\tau''} \quad \text{and} \quad \sigma^2 = \left(\frac{1}{\Delta\tau'} + \frac{1}{\Delta\tau''}\right)^{-1}
$$

Now what to do with this next? From expression above, instead accept or reject move, we just sample $x_k$ from the gaussian distribution, no rejection!

In [ ]:
using Distributions

"""
Construct levy free density matrix
"""
function levy_free(x_bound,β,N)
    Δτ = β / N
    x = [x_bound[1]]
    xN = x_bound[2]
    for k in 2:N
        Δτ_rel = (N - k) * Δτ
        μ = (Δτ_rel * x[k-1] + Δτ * xN) / (Δτ_rel + Δτ)
        σ2 = inv(Δτ_rel) + inv(Δτ)
        σ2 = inv(σ2)
        σ = sqrt(σ2)
        gauss = Normal(μ,σ)
        x_new = rand(gauss)
        push!(x,x_new)
    end
    push!(x,xN)
    return x
end;

In [ ]:
#plot path
β = 4
N = 10_000
x = levy_free([0.,0.],β,N);

p = plot(framestyle=:box,size=(300,400))
plot!(p,x,0:length(x)-1,label="")
yticks!(p,[0,N],["0","\\beta"])
ylims!(p,0,length(x)-1)
xlims!(p,-2,2)
ylabel!(p,"\\tau")
xlabel!(p,"x")

### Fourier Space

#### Continous Path

By analyzing the geometry of the path, we can develop new algorithm of path integral based on its fourier transform. For simplification, we consider $x(0) = x(\beta) = 0$.
Other path $x(\tau)$ can be decomposed as set of sine function,

$$
x(\tau) = \sum_{n=1}^\infty c_n \sin \left(n\pi \frac{\tau}{\beta}\right); \quad \tau \in [0,\beta]
$$

we can get the fourier coefficient $c_l$ from orthonormality relation, hence by multiply the equation above with $ \sin \left(l\pi \frac{\tau}{\beta}\right)$ and integrate it

$$
\frac{2}{\beta} \int_0^\beta d\tau \sin \left(l\pi\frac{\tau}{\beta}\right) x(\tau) = \frac{2}{\beta}\sum_{n=1}^\infty c_n \int_0^\beta d\tau \sin \left(l\pi\frac{\tau}{\beta} \right) \sin \left(n\pi\frac{\tau}{\beta}\right) = c_l 
$$

We now express the statistical weight of the path directly in fourier variables,

$$
\exp(-\mathcal{S}(x_0,x_1,...,x_N) = \rho_0(x_0,x_1,\Delta\tau)\rho_0(x_1,x_2,\Delta\tau)\dots\rho_0(x_{N-1},x_N,\Delta\tau)
$$

for small $\Delta\tau$, in this limit the action becomes

$$
\mathcal{S} = \frac{1}{2} \int_0^\beta d\tau \left[\frac{\partial x(\tau)}{\partial \tau}\right]^2 = \frac{1}{2} \sum_{n=1}^\infty c^2_n\frac{n^2\pi^2}{\beta^2}\int_0^\beta d\tau \cos^2\left(n\pi\frac{\tau}{\beta}\right) = \frac{1}{\beta}\sum_{n=1}^\infty \frac{c_n^2n^2\pi^2}{4}
$$

the statistical weight of the path

$$
\exp(-\mathcal{S}) = \exp\left( - \frac{1}{\beta}\sum_{n=1}^\infty \frac{c_n^2n^2\pi^2}{4} \right)
$$

thus free density matrix is an infinite product of integrals

$$
\rho_0(0,0,\beta) \propto \prod_{n=1}^\infty \left[ \int_{-\infty}^\infty \frac{dc_n n\pi }{\sqrt{4\pi \beta}} \exp\left(-\frac{1}{\beta}\sum_{n=1}^\infty c_n^2\frac{n^2\pi^2}{4}\right)\right]
$$

#### Discrete path
A discrete path can be represented as finite mode of fourier transform:

$$
x_k = \sum_{n=1}^{N-1} c_n\sin\left(n\pi \frac{k}{N}\right); \quad k=0,\dots,N
$$

the action $\mathcal{S}$ is (by taking $\Delta\tau$ as discrete)

$$
\exp\left[-\mathcal{S}\right] = \exp\left[\sum_{k=1}^N\frac{(x_k - x_{k-1})^2}{2\Delta\tau}\right]
$$

because of orthogonility, term $n \neq l$ vanish, and we end up with

$$
\mathcal{S} = \frac{1}{2\Delta\tau}\sum_{j=1}^{N=1}c_j^2\sum_{k=1}^N\left[\sin\left(j\pi\frac{k}{N}\right)-\sin\left(j\pi\frac{k-1}{N}\right)\right]^2 \\ 
= \frac{2}{\Delta\tau}\sum_{n=1}^{N-1}c_n^2\sin^2\left(\frac{n\pi}{2N}\right)\sum_{k=1}^N \cos^2\left[\frac{n\pi}{N}\left(k-\frac{1}{2}\right)\right] = -\frac{N}{\Delta\tau} \sum_{n=1}^{N-1}c_n^2\sin^2\left(\frac{n\pi}{2N}\right)
$$

writing weight as gaussian

$$
\exp(\mathcal{S}) = \exp\left(-\frac{c_1^2}{2\sigma_1^2}\right) \dots \exp\left(-\frac{c_{N-1}^2}{2\sigma_{N-1}^2}\right); \quad \sigma_n^2 = \frac{\beta}{2N^2\sin^2\left(\frac{n\pi}{2N}\right)} \approx \frac{2\beta}{\pi^2n^2} + \dots
$$

the path integral representation is purely independent in fourier transform

$$
\rho_0(0,0,\beta) = \prod_{i=1}^{N-1} \int_{-\infty}^\infty \frac{dc_i}{\sqrt{2\pi}\sigma_i}\exp\left(-\frac{c_i^2}{2\sigma_i^2}\right)
$$

now let's sampling using fourier path integral

In [ ]:
"""
fourier free path

Sampling a path contributing to ρ_0 (0,0,β) in fourier space, and then transforming it to real variable.

Step:
# for every fourier mode n ∈ [1,N-1], sample fourier coeff
for i in 1:N-1
    1. calculate standard deviation, σ
    2. sample fourier coeff, c_i, from gaussian with σ
# transform fourier to real space, {c_1,...,c_N} → {x_1,...,x_k}
for k in 0:N
    1. calculate x_k from fourier coeff

return: {x_1,...,x_k}
"""
function fourier_free_path(β,N)
    σ = zeros(Float64,N-1)
    c = zeros(Float64,N-1)
    for i in 1:N-1
        σ[i] = 2N^2 * sin(i*π/(2N))^2
        gauss = Normal(0,sqrt(β/σ[i]))
        c[i] = rand(gauss)
    end
    x = zeros(Float64,N+1)
    for k in 0:N
        ns = LinRange(1,N-1,N-1)
        s = sin.(ns .* π*k/N)
        x[k+1] = sum(c .* s)
    end
    x[end] = x[1]
    return x
end

β = 4
N = 10_000
x = fourier_free_path(β,N)

using Plots
p = plot(framestyle=:box,size=(300,400))
plot!(p,x,0:length(x)-1)
yticks!(p,[0,N],["0","\\beta"])
ylims!(p,0,length(x)-1)
xlims!(p,(-4,4))
ylabel!(p,"\\tau")
xlabel!(p,"x")

Fourier transform decouples the real-action because of the latter is invariant under translation. In many other systems, simulations can be difficult within a set of coordinate, and much easier after a coordinate transformations, because the variables might be less coupled.

### Correlation Function

We now determine all path correlation $\left\langle x_k x_l \right\rangle$ from path integral action. Using $\Delta\tau = 1$, so that $N = \beta$, the action is

$$
\mathcal{S} = \frac{1}{2}[(x_1 - x_0)^2 + \dots + (x_N - x_{N-1})^2]
$$

this action/weight can be represented as matrix

$$
\exp(-\mathcal{S}) = \exp\left(-\frac{1}{2} \sum_{k,l=1}^{N-1} x_k \mathcal{M}_{kl} x_l \right)
$$

where

$$
\mathcal{M} = \begin{bmatrix}
2 & -1 & \cdot & \cdot & \cdot & \cdot & \cdot \\
-1 & 2  & -1 & \cdot & \cdot & \cdot & \cdot \\
\cdot & -1 & 2  & -1 & \cdot & \cdot & \cdot \\
\cdot & \cdot & -1 & 2  & -1 & \cdot & \cdot \\
\cdot & \cdot & \cdot & -1 & 2  & -1 & \cdot \\
\cdot & \cdot & \cdot & \cdot & -1 & 2  & -1 \\
\cdot & \cdot & \cdot & \cdot & \cdot & -1 & 2 \\
\end{bmatrix}
$$

the correlation function is given by
$$
\left\langle x_k x_l \right\rangle = \frac{\int dx_1 \dots dx_{N-1} x_k x_l \exp\left(-\frac{1}{2} \sum x_n \mathcal{M}_{nm} x_m \right)}{\int dx_1 \dots dx_{N-1} \exp\left(-\frac{1}{2} \sum x_n \mathcal{M}_{nm} x_m \right)} = (\mathcal{M}^{-1})_{kl} 
$$

where it is simply a inverse of matrix $\mathcal{M}$